In [ ]:
# Importing Dependencies.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
##########################
import pandas as pd
import numpy as np
#########################
from scipy import stats
#########################
import cufflinks as cf
cf.go_offline()
#########################
import datetime as dt
from datetime import timedelta
##########################
import matplotlib.pyplot as plt
import pyfolio as pf
import empyrical as ep

In [ ]:
from tvDatafeed import TvDatafeed, Interval
##########################################
username = ''
password = ''
##########################################
tv = TvDatafeed(username, password)
# NIFTY INDEX.
Nifty_index_data = tv.get_hist(symbol='NIFTY',
                                    exchange='NSE',
                                    interval=Interval.in_1_minute,
                                    n_bars=20000)
Nifty_index_data = df = Nifty_index_data[['symbol','open','high','low','close','volume']]
df = df.between_time('9:05', '15:00')

In [ ]:
import numpy as np
from scipy import stats

def linear_regression_backtest(data, lookback_period):
    # Check if data has 'close' column
    if 'close' not in data.columns:
        raise ValueError("Input dataframe must have 'close' column")

    # Check if lookback period is valid
    if len(data) < lookback_period:
        raise ValueError("Lookback period is larger than available data")

    # Calculate the linear regression slope for each period in the dataset
    data['slope'] = np.nan
    for i in range(lookback_period, len(data)):
        # Select the data for the lookback period
        x = np.arange(lookback_period)
        y = data['close'][i-lookback_period+1:i+1].values
        # Calculate the slope of the linear regression line
        slope, _, _, _, _ = stats.linregress(x, y)
        # Add the slope to the data frame
        data.loc[data.index[i], 'slope'] = slope

    # Determine the trading signals based on the slope of the linear regression line
    data['signal'] = np.where(data['slope'] > 0, 1, -1)
    # Commented out: alternate way of determining trading signals
    # data['signal'] = np.where(data['slope'] < 0, -1, data['signal'])

    ### Determine the trading signals based on the slope of the linear regression line ###
    # Commented out: another alternate way of determining trading signals
    # data['signal'] = data['signal'].where(data['signal'].diff().fillna(0).ne(0), np.nan)
    # data.loc[(data.index.time >= pd.to_datetime('15:15:00').time()) , 'signal'] = 0

    # Calculate the daily returns based on the trading signals
    data = data.sort_index() # ensure data is sorted by date
    data['returns'] = data['close'].pct_change()
    data['strategy_returns'] = data['returns'] * data['signal']
    
    return data


In [ ]:
# Load in a dataset of closing prices for a stock
data = df

In [ ]:
# Backtest the linear regression indicator with a lookback period of 20 periods
df = linear_regression_backtest(data, 3)

In [ ]:
# Calculate the annualised Sharpe ratio
sharpe = np.sqrt(252) * \
    df.strategy_returns.mean() / df.strategy_returns.std()
# Calculating Max Drawdown.
max_dd= ep.max_drawdown(df.strategy_returns)
# Calculating Stablity of Time Series of Returns.
stablity = ep.stability_of_timeseries(df.strategy_returns)

In [ ]:
print(f'Start: \n{df.index[0]} \nEnd: \n{df.index[-1]}')
print("The annualised Sharpe ratio is %.2f" % sharpe)
print(f'Total Signal Counts:\n{df.signal.value_counts()}')
print(f'Total return: {((df.strategy_returns.cumsum()[-1])) :.4f} X') #times X
print(f'The Max Drawdown: {(max_dd*100):.2f} %') #times 100
print(f'Stability of Time Series: {stablity:.2f}')

In [ ]:
(df['close']/df['close'][0]).iplot()

##
- SLOPE is 2 SIGNAL COUNT = 1//1786 , -1//1785

- SLOPE is 3 SIGNAL COUNT = 1//1146 , -1//1146

- SLOPE is 4 SIGNAL COUNT = 1//839 , -1//839 

- TF45MIN
##

In [ ]:
pf.create_full_tear_sheet(df.strategy_returns)

In [ ]:
j = df.signal  # forward-fill missing values in df.position
e = df.close

result = []
for i in range(len(j)):
    if i == len(j)-1 or pd.isna(j[i]) or pd.isna(j[i+1]):
        result.append(0)  # if there's a missing value or last row, append 0
    elif j[i] == -1:
        result.append(e[i] - e[i+1])
    elif j[i] == 1:
        result.append(e[i+1] - e[i])
    else:
        result.append(0)

total = sum(result)
print(f'Gains in Points: {total:.2f}')


In [ ]:
df

df = df.to_csv('slope4-45m--ET-15.15-regren.csv')

df['slope'].describe()